# LA Bikeshare Data
After working on my [Google Location History Notebook](https://nbviewer.jupyter.org/github/black-tea/google_location_history/blob/master/MyTimeinLA.ipynb), I wanted to dig into spatial data a bit more. I've seen a number of different bikeshare data analyses in the past few years; I thought I would give it a try myself.
### Extract: Get the data from LA Metro
Go to https://bikeshare.metro.net/about/data/ and download all the (1) trip data and (2) station information. As of 7/15/2017, there is one year of trip data released, separated by quarters. Everything is in CSV format.

In [45]:
##### Setup
%matplotlib inline
import urllib2
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pysal as ps
import matplotlib.pyplot as plt
import datetime
import folium

##### About LA Metro Bikeshare Data
Attribute information is provided directly on the [LA Metro website](https://bikeshare.metro.net/about/data/). The station information contains the following fields:
* Station ID: Unique identifier for the station
* Station Name: Name of the station
* Go Live Date: Date that the station first became active
* Status: "Active" for stations available or "Inactive" for stations that are not available as of the latest update

The trip data contains the following fields:
* Trip ID ("trip_id"): Unique identifier for the trip
* Duration ("duration"): Duration of the trip, in minutes
* Start Time ("start_time"): Date/Time that the trip began, in ISO 8601 format in local time
* End Time ("end_time"): Date/Time that the trip ended, in ISO 8601 format in local time
* Start Station ("start_station"): Station ID where the trip originated
* Start Latitude ("start_lat"): Y coordinate of the station where the trip originated
* Start Longitude ("start_lon"): X coordinate of the station where the trip originated
* End Station ("end_station"): Station ID where the trip terminated
* End Latitude ("end_lat"): Y coordinate of the station where the trip terminated
* End Longitude ("end_lon"): X coordinate of the station where the trip terminated
* Bike ID ("bike_id"): Unique identifier for the bike
* Plan Duration ("plan_duration"): number of days that the plan the passholder is using entitles them to ride; 0 is used for a single ride plan (Walk-up)
* Trip Type ("trip_route_category"): One Way or Round Trip
* Passholder Type ("passholder_type"): Name of the passholders plan

#### Where is Metro Bikeshare?
I wanted to start by looking at the stations on a map. However, since the station information table doesn't contain lat/lon coordinates, I would need to merge the tale with the trip data OR use the [updated GeoJSON feed](https://bikeshare.metro.net/stations/json/) on the LA Metro website. I opted for the latter. 

In [46]:
# Initially getting HTTP Forbidden Error, so added headers
url = "https://bikeshare.metro.net/stations/json/"
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

# Load the GeoJSON
req = urllib2.Request(url, headers=hdr)
try:
    response = urllib2.urlopen(req)
except urllib2.HTTPError, e:
    print e.fp.read()
station_json = json.loads(response.read())

# Import to GeoDataFrame
stations = gpd.GeoDataFrame.from_features(station_json['features'])

# Create basemap and add station points
station_map = folium.Map([34.047677, -118.3073917], tiles='CartoDB positron', zoom_start=11)
for index, row in stations.iterrows():
    folium.Marker(
            [str(row.geometry.centroid.y), str(row.geometry.centroid.x)],
            popup=row['name']
            ).add_to(station_map)
station_map

The system only exists downtown and (as of only very recently) in Pasadena. We will come back to this map later. Let's turn our attention to trip data.
#### What does the LA Metro Bikeshare Ridership look like?
To answer this question, I'm going to join together the trip data provided by Metro (4 quarters as of 7/17/2017). Once we load in the trip data to get a sense for the data structure, let's get a sense for LA bikeshare ridership.

In [ ]:
# Load the station file
station_path = 'data/metro_station_table.csv'
stations = pd.read_csv(station_path)

# Load the trip data
q1 = pd.read_csv('data/la_metro_gbfs_trips_Q1_2017.csv')
q2 = pd.read_csv('data/la_metro_gbfs_trips_Q2_2017.csv')
q3 = pd.read_csv('data/MetroBikeShare_2016_Q3_trips.csv')
q4 = pd.read_csv('data/Metro_trips_Q4_2016.csv')

# Concatenate data
frames = [q1,q2,q3,q4]
trips = pd.concat(frames)
stations.head()

,Station ID,Station Name,Go live date,Status
0,3000,Virtual Station,2016-07-07,Active
1,3005,Flower & 7th,2016-07-07,Active
2,3006,Olive & 8th,2016-07-07,Active
3,3007,5th & Grand,2016-07-07,Active
4,3008,Figueroa & 9th,2016-07-07,Active


In [ ]:
# Begin by converting the "start_time" and "end_time" columns into datetime64 columns, add a count column
trips['start_time'] = pd.to_datetime(trips['start_time'], errors='raise', infer_datetime_format='True')
trips['end_time'] = pd.to_datetime(trips['end_time'], errors='raise', infer_datetime_format='True')
trips['count'] = 1

# Aggregate by month / year along the count column
per_month = trips.start_time.dt.to_period("M")
g = trips.groupby(per_month)
ridership_by_month = g.sum()['count']

# Create a bar chart showing the percentage of time spent doing each activity
ridership_chart = ridership_by_month.plot(kind='bar', title='Number of Trips each Month', figsize=(15,10), fontsize=12)
ridership_chart.set_xlabel("Year - Month", fontsize=12)
ridership_chart.set_ylabel("Trips", fontsize=12)
plt.show()

There are about 15k - 20k trips per month for the bikeshare system. As expected, ridership dips down in the so-called winter months and ticks back up inthe spring and summer. 

The high mark for the program was the second month of operation, August 2016. Why not the first month? The system didn't launch until July 7, 2016, which meant that July was missing a quarter month of ridership. 

Another reason for the August peak: Metro had an [introductory 50 percent discount rate](http://thesource.metro.net/2016/07/07/its-official-metro-bike-share-program-launches-in-dtla/) that was offered August - September 2016, so walk-up trips that were normally 3.50 were discounted to 1.75. I know this because I took advantage of it several times.

#### LA Metro Ridership by Station
Let's look at the differences in ridership by station, separating origins and destinations.

In [ ]:
# First reformat stations df by setting station id as the index
stations = stations.set_index('Station ID')

# Count origins and destinations
stations_o_count = trips['count'].groupby(trips['start_station_id']).count().to_frame('o_ct')
stations_d_count = trips['count'].groupby(trips['end_station_id']).count().to_frame('d_ct')

# Combine O/D counts with station information
stations_od_count = pd.merge(stations_o_count, stations_d_count, right_index=True, left_index=True)
stations_count = pd.merge(stations, stations_od_count, left_index=True, right_index=True)
stations_count


In [ ]:
what is the density of bikeshare stations?